In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/My Drive/Colab Notebooks/CV2/Project

/content/drive/My Drive/Colab Notebooks/CV2/Project


In [7]:
import numpy as np
from PIL import Image
import os

# Function to calculate MSE between two images
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    return err

def load_images(filepaths):
    images = []
    for filepath in filepaths:
        image = Image.open(filepath).convert('L')
        images.append(np.array(image))
    return images

def calculate_loss(images, i):
    # Note that we need to check if i-1 and i+1 are within the valid range of the images
    loss = 0
    if 0 <= i-1 < len(images):
        loss += mse(images[i], images[i-1])
    if 0 <= i+1 < len(images):
        loss += mse(images[i], images[i+1])
    prev_input = (i // 4) * 4 - 4
    if prev_input == 6:
      prev_input = prev_input - 4

    next_input = prev_input + 4

    print(next_input)

    loss -= mse(images[i], images[prev_input])
    loss -= mse(images[i], images[next_input])


    return loss



In [9]:
folder_path = "data/frame_seq" # Replace <FOLDER_PATH> with the actual folder path input by the user

# Generate the list of file paths for each set of images assuming they are named consistently as imX_Y.png
# where X is the sequence number and Y is the frame number in that sequence.
file_suffixes = ['0.000000', '0.250000', '0.500000', '0.750000']
file_indices = [1, 2, 3, 4, 5, 6]
image_filepaths = [
    f"{folder_path}/im{index}_{suffix}.png"
    for index in file_indices for suffix in file_suffixes
]
image_filepaths.append("data/frame_seq/im7_0.000000.png")

# This function will be modified to accept folder_path as an input, and will check for file presence based on that
def check_files_and_calculate_loss():
    # Check if all the files are present in the given folder path
    all_files_present = all(os.path.exists(filepath) for filepath in image_filepaths)

    if all_files_present:
        images = load_images(image_filepaths)

        # Calculate losses for each image
        losses = [calculate_loss(images, i) for i in range(len(images))]

        # Pair each loss with its corresponding image index
        indexed_losses = list(enumerate(losses))

        # Sort the losses in descending order
        indexed_losses.sort(key=lambda x: x[1], reverse=True)

        # Now we apply the restriction that no more than two frames are interpolated from any given pair of input frames
        selected_frames = []
        counts = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}  # Keep track of how many times a frame from each input has been selected

        for index, loss in indexed_losses:
            frame_index = (index // 4) + 1  # Determine the input frame index based on sorted index
            if counts[frame_index] < 1:  # Apply the restriction
                selected_frames.append((image_filepaths[index], loss))
                counts[frame_index] += 1

        return selected_frames
    else:
        return "Not all image files are present in the provided folder path"

output = check_files_and_calculate_loss()

0
0
0
0
4
4
4
4
8
8
8
8
12
12
12
12
16
16
16
16
20
20
20
20
24


In [10]:
output

[('data/frame_seq/im2_0.000000.png', -52.41964285714285),
 ('data/frame_seq/im6_0.000000.png', -89.70246233258928),
 ('data/frame_seq/im5_0.000000.png', -92.22883823939732),
 ('data/frame_seq/im3_0.000000.png', -96.31934465680804),
 ('data/frame_seq/im7_0.000000.png', -106.35861642020089),
 ('data/frame_seq/im4_0.000000.png', -109.5898001534598),
 ('data/frame_seq/im1_0.750000.png', -854.606672014509)]

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def calculate_psnr_for_images(target_image_path, images_folder_path):
    """
    Calculate the PSNR between each image in the given folder and the target image.

    Parameters:
    target_image_path (str): The file path to the target image.
    images_folder_path (str): The folder path where the other images are stored.

    Returns:
    dict: A dictionary with the filename as the key and the corresponding PSNR as the value.
    """
    # Load the target image
    target_image = Image.open(target_image_path).convert('L')
    target_image = np.array(target_image)

    image_files = [f for f in os.listdir(images_folder_path) if os.path.isfile(os.path.join(images_folder_path, f))]

    # Calculate PSNR for each image and store in a dictionary
    psnr_results = {}
    for image_file in image_files:
        # Load the image
        image_path = os.path.join(images_folder_path, image_file)
        image = Image.open(image_path).convert('L')
        image = np.array(image)

        # Calculate PSNR
        psnr_value = compare_psnr(target_image, image)

        # Store the PSNR in the dictionary
        psnr_results[image_file] = psnr_value

    return psnr_results

# Placeholder function call - replace the strings with actual paths provided by the user
# psnr_results = calculate_psnr_for_images("<TARGET_IMAGE_PATH>", "<IMAGES_FOLDER_PATH>")


psnr_results = calculate_psnr_for_images("data/ground_truth/im4.png", "data/frame_seq")


In [ ]:
psnr_results
max_psnr_image = max(psnr_results, key=psnr_results.get)
max_psnr_value = psnr_results[max_psnr_image]
print(max_psnr_value)

35.7882974975701
